In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests
from config import weather_api_key
from datetime import datetime

In [2]:
# Generate a new set of 1,500 random latitudes and longitudes
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
coordinates = list(lat_lngs)

In [3]:
# Get the nearest city using the citipy module
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)

In [4]:
# Perform an API call with the OpenWeatherMap
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

# Create an empty list to hold the weather data.
city_data = []

# Loop through all the cities in the list.
for city in cities:
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city
    # Run an API request for each of the cities.
    try:
        city_weather = requests.get(city_url).json()
        
        # City country
        city_country = city_weather["sys"]["country"]

        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')

        # Latitude and longitude
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        
        # Maximum temperature
        city_max_temp = city_weather["main"]["temp_max"]
        
        # Percent humidity
        city_humidity = city_weather["main"]["humidity"]
        
        # Percent cloudiness
        city_clouds = city_weather["clouds"]["all"]
        
        # Wind speed
        city_wind = city_weather["wind"]["speed"]
        
        # Weather description
        city_description = city_weather["weather"][0]["description"]
        
        # Rain fall in the past 3 hours
        try: 
            city_rain = city_weather["rain"]["3h"]
        except:
            city_rain = 0 
            
        # Snow fall in the past 3 hours
        try:
            city_snow = city_weather["snow"]["3h"]
        except:
            city_snow = 0
            
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Date": city_date,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_description,
                          "Rain inches (last 3 hours)": city_rain,
                          "Snow inches (last 3 hours)": city_snow
                         })

    # If an error is experienced, skip the city.
    except:
        pass

In [5]:
# Add the data to a new DataFrame
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hours),Snow inches (last 3 hours)
0,Punta Arenas,CL,2020-02-23 21:17:15,-53.15,-70.92,55.40,62,75,32.21,broken clouds,0.0,0.0
1,Hermanus,ZA,2020-02-23 21:19:29,-34.42,19.23,65.71,75,0,26.80,clear sky,0.0,0.0
2,Castro,BR,2020-02-23 21:19:29,-24.79,-50.01,67.93,78,90,2.66,overcast clouds,0.0,0.0
3,Albany,US,2020-02-23 21:16:59,42.60,-73.97,51.80,29,75,5.82,broken clouds,0.0,0.0
4,Okhotsk,RU,2020-02-23 21:19:29,59.38,143.30,-14.96,77,16,7.90,few clouds,0.0,0.0
5,Buraydah,SA,2020-02-23 21:19:29,26.33,43.98,68.00,49,30,9.17,scattered clouds,0.0,0.0
6,Mataura,NZ,2020-02-23 21:18:08,-46.19,168.86,60.01,80,77,8.01,broken clouds,0.0,0.0
7,Airai,TL,2020-02-23 21:17:33,-8.93,125.41,66.69,82,0,4.43,clear sky,0.0,0.0
8,Kapaa,US,2020-02-23 21:19:30,22.08,-159.32,79.00,69,1,8.05,clear sky,0.0,0.0
9,Ushuaia,AR,2020-02-23 21:19:30,-54.80,-68.30,46.40,87,75,21.92,light rain,0.0,0.0


In [10]:
# Save the new DataFrame as a CSV file
output_data_file = "data/WeatherPy_challenge.csv"

In [11]:
# Upload the CSV file as part of your submission as WeatherPy_challenge.csv
city_data_df.to_csv(output_data_file, index_label="City_ID")

In [8]:
# How many cities have recorded rainfall or snow
cities_with_rain = city_data_df[(city_data_df["Rain inches (last 3 hours)"] > 0)]["City"].count()
cities_with_rain

52

In [9]:
# How many cities have recorded snow
cities_with_snow = city_data_df[(city_data_df["Snow inches (last 3 hours)"] > 0)]["City"].count()
cities_with_snow

28